In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import bioformats.omexml as ome
import tifffile
import sys
import argparse
import os
import logging
import tensorflow as tf
from skimage import exposure#, external
from scipy import ndimage, signal, stats
from math import floor, ceil
from flowdec import data as fd_data
from flowdec import psf as fd_psf
from flowdec import restoration as fd_restoration
from xml.etree import cElementTree as ElementTree
from tqdm.notebook import tqdm
#from tensorflow.python.compiler.mlcompute import mlcompute; 
#mlcompute.set_mlc_device(device_name='gpu')

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')


Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
ls Z:\Shared243\storal\2024-01-30_mc191\test_scott

 Volume in drive Z is SYSTEM
 Volume Serial Number is ECF6-6273

 Directory of Z:\Shared243\storal\2024-01-30_mc191\test_scott

19/02/2024  16:55    <DIR>          .
19/02/2024  16:40    <DIR>          ..
19/02/2024  16:56     3,090,910,931 slide2_deskewed_cropped.tif
               1 File(s)  3,090,910,931 bytes
               2 Dir(s)  24,720,320,059,392 bytes free


In [3]:
input_file_str = "Z:/Shared243/storal/2024-01-30_mc191/test_scott/slide2_deskewed_cropped.tif"
dat = tifffile.imread(input_file_str)

In [4]:
bead640_image_file_str = "Z:/Shared243/storal/2024-01-30_mc191/640PSF_sigma.csv"
psf640 = np.genfromtxt(bead640_image_file_str, delimiter=',')
bead488_image_file_str = "Z:/Shared243/storal/2024-01-30_mc191/488PSF_sigma.csv"
psf488 = np.genfromtxt(bead488_image_file_str, delimiter=',')

In [5]:
kernel_shape = (51,51,51)
kernel488 = np.zeros(kernel_shape) #(51,51,51)) #Note may not work if this size is too big relative to the image
for offset in [0,1]:
    kernel488[tuple((np.array(kernel488.shape) - offset) // 2)] = 1
#assume bead image at different resolution in z to actual data, and difference is a factor of 3.08 (308nm versus 100nm)
#estimated and stored 3d psf, but assume here diagonal psf. Could adjust this, but already very close to diagonal
kernel488 = ndimage.gaussian_filter(kernel488, sigma=[np.sqrt(psf488[2,2])/2.705078,np.sqrt(psf488[0,0]),np.sqrt(psf488[1,1])])

kernel640 = np.zeros(kernel_shape) #(51,51,51)) #Note may not work if this size is too big relative to the image
for offset in [0,1]:
    kernel640[tuple((np.array(kernel640.shape) - offset) // 2)] = 1
#assume bead image at different resolution in z to actual data, and difference is a factor of 3.08 (308nm versus 100nm)
#estimated and stored 3d psf, but assume here diagonal psf. Could adjust this, but already very close to diagonal
kernel640 = ndimage.gaussian_filter(kernel640, sigma=[np.sqrt(psf640[2,2])/2.705078,np.sqrt(psf640[0,0]),np.sqrt(psf640[1,1])])

In [26]:
data = np.copy(dat) 
res = np.zeros(data.shape)
ndim = 3 #data.ndim 
algo = fd_restoration.RichardsonLucyDeconvolver(ndim, pad_mode='none', pad_min=[32,32,32]).initialize() 

In [27]:
dat.shape

(284, 59, 2, 210, 219)

In [28]:
SizeT=dat.shape[0]
range(SizeT)

range(0, 284)

In [29]:
for i,image in enumerate(data):
    print(i)
    print(image.shape)
    tt488 = image[:,0 ]
    print(tt488.shape)

0
(59, 2, 210, 219)
(59, 210, 219)
1
(59, 2, 210, 219)
(59, 210, 219)
2
(59, 2, 210, 219)
(59, 210, 219)
3
(59, 2, 210, 219)
(59, 210, 219)
4
(59, 2, 210, 219)
(59, 210, 219)
5
(59, 2, 210, 219)
(59, 210, 219)
6
(59, 2, 210, 219)
(59, 210, 219)
7
(59, 2, 210, 219)
(59, 210, 219)
8
(59, 2, 210, 219)
(59, 210, 219)
9
(59, 2, 210, 219)
(59, 210, 219)
10
(59, 2, 210, 219)
(59, 210, 219)
11
(59, 2, 210, 219)
(59, 210, 219)
12
(59, 2, 210, 219)
(59, 210, 219)
13
(59, 2, 210, 219)
(59, 210, 219)
14
(59, 2, 210, 219)
(59, 210, 219)
15
(59, 2, 210, 219)
(59, 210, 219)
16
(59, 2, 210, 219)
(59, 210, 219)
17
(59, 2, 210, 219)
(59, 210, 219)
18
(59, 2, 210, 219)
(59, 210, 219)
19
(59, 2, 210, 219)
(59, 210, 219)
20
(59, 2, 210, 219)
(59, 210, 219)
21
(59, 2, 210, 219)
(59, 210, 219)
22
(59, 2, 210, 219)
(59, 210, 219)
23
(59, 2, 210, 219)
(59, 210, 219)
24
(59, 2, 210, 219)
(59, 210, 219)
25
(59, 2, 210, 219)
(59, 210, 219)
26
(59, 2, 210, 219)
(59, 210, 219)
27
(59, 2, 210, 219)
(59, 210, 219)
28

In [30]:
niter = 5
for i,image in tqdm(enumerate(data), total=data.shape[0], desc='Deconvolving: '):
    tt488 = image[:,0]
    nonzero = tt488[np.where(tt488>0)]
    bkgd_mode = stats.mode(nonzero)[0][0]
    bkgd_std = stats.tstd(nonzero)
    indz,indy,indx = np.where(tt488==0)
    tt488[indz, indy, indx] = bkgd_mode + bkgd_std*np.random.randn(len(indz))
    # Note that deconvolution initialization is best kept separate from 
    # execution since the "initialize" operation corresponds to creating 
    # a TensorFlow graph, which is a relatively expensive operation and 
    # should not be repeated across iterations if deconvolving more than 
    # one image
    res[i, :, 0] = algo.run(fd_data.Acquisition(data=tt488, kernel=kernel488), niter=niter).data

    tt640 = image[:,1]
    nonzero = tt640[np.where(tt640>0)]
    bkgd_mode = stats.mode(nonzero)[0][0]
    bkgd_std = stats.tstd(nonzero)
    indz,indy,indx = np.where(tt640==0)
    tt640[indz, indy, indx] = bkgd_mode + bkgd_std*np.random.randn(len(indz))
    # Note that deconvolution initialization is best kept separate from 
    # execution since the "initialize" operation corresponds to creating 
    # a TensorFlow graph, which is a relatively expensive operation and 
    # should not be repeated across iterations if deconvolving more than 
    # one image
    res[i, :, 1] = algo.run(fd_data.Acquisition(data=tt640, kernel=kernel640), niter=niter).data


Deconvolving:   0%|          | 0/284 [00:00<?, ?it/s]

In [13]:
with tifffile.TiffFile(input_file_str) as tif:
        mdata = tif.imagej_metadata
mdata

{'ImageJ': '1.54f',
 'images': 33512,
 'channels': 2,
 'slices': 59,
 'frames': 284,
 'hyperstack': True,
 'mode': 'grayscale',
 'unit': 'micron',
 'spacing': 0.2705078125,
 'loop': False,
 'min': 93.0,
 'max': 123.0,
 'Info': ' BitsPerPixel = 16\n DimensionOrder = XYCZT\n IsInterleaved = true\n IsRGB = false\n LittleEndian = true\n PixelType = uint16\n Series 0 Name = Capture 1 DR\n SizeC = 2\n SizeT = 285\n SizeX = 754\n SizeY = 512\n SizeZ = 59\nLocation = Z:\\Shared243\\storal\\2024-01-30_mc191\\slide2_deskewed.sld\n',
 'Labels': ['c:1/2 z:1/59 t:1/285 - Capture 1 DR',
  'c:2/2 z:1/59 t:1/285 - Capture 1 DR',
  'c:1/2 z:2/59 t:1/285 - Capture 1 DR',
  'c:2/2 z:2/59 t:1/285 - Capture 1 DR',
  'c:1/2 z:3/59 t:1/285 - Capture 1 DR',
  'c:2/2 z:3/59 t:1/285 - Capture 1 DR',
  'c:1/2 z:4/59 t:1/285 - Capture 1 DR',
  'c:2/2 z:4/59 t:1/285 - Capture 1 DR',
  'c:1/2 z:5/59 t:1/285 - Capture 1 DR',
  'c:2/2 z:5/59 t:1/285 - Capture 1 DR',
  'c:1/2 z:6/59 t:1/285 - Capture 1 DR',
  'c:2/2 z

In [31]:
suffix = f"_scottdecon_niter{niter}_padding32.tif"
suffix

'_scottdecon_niter5_padding32.tif'

In [32]:
output_file_str = input_file_str.replace(".tif", suffix)

In [33]:
output_file_str

'Z:/Shared243/storal/2024-01-30_mc191/test_scott/slide2_deskewed_cropped_scottdecon_niter5_padding32.tif'

In [34]:
####################
# print('Done. Now writing to file\n')
# # create numpy array with correct order
# if is_single_time_pt:
#     img5d = np.expand_dims(np.expand_dims(res, axis=1), axis=0) 
# else:
#     img5d = np.expand_dims(res, axis=1)
# # write file and save OME-XML as description
tifffile.imwrite(output_file_str, res.astype(np.uint16), imagej = True, metadata=mdata)
print('All finished\n')

All finished



In [8]:
dat

array([[[[[112, 102, 102, ..., 103, 105, 105],
          [102, 102, 103, ..., 100, 101,  99],
          [102, 101, 103, ..., 103, 104, 107],
          ...,
          [102, 102, 100, ..., 104, 102, 104],
          [101, 103, 101, ..., 100, 101, 101],
          [103, 103, 104, ..., 102, 103, 102]],

         [[ 98, 101, 102, ..., 100, 100,  99],
          [101, 101, 103, ..., 101, 101, 100],
          [102, 104, 104, ..., 101, 100, 103],
          ...,
          [104, 102, 103, ..., 104, 103, 104],
          [ 98, 106, 103, ..., 100,  96, 104],
          [106, 103,  97, ..., 104, 105,  99]]],


        [[[101, 104, 101, ..., 102, 104, 107],
          [104, 104, 106, ..., 104, 100,  99],
          [102, 104, 101, ..., 102,  99, 101],
          ...,
          [102, 104, 101, ..., 103, 100, 102],
          [100,  98, 104, ..., 102, 101, 104],
          [103, 106, 102, ..., 101, 104, 104]],

         [[100, 101, 105, ...,  97, 104, 102],
          [103, 102, 107, ..., 103, 103, 105],
       

In [ ]:


dat = tifffile.imread(input_file_str)
psf = np.genfromtxt(bead_image_file_str, delimiter=',')
print("Data successfully read in \n")

kernel = np.zeros(kernel_shape) #(51,51,51)) #Note may not work if this size is too big relative to the image
for offset in [0,1]:
    kernel[tuple((np.array(kernel.shape) - offset) // 2)] = 1
#assume bead image at different resolution in z to actual data, and difference is a factor of 3.08 (308nm versus 100nm)
#estimated and stored 3d psf, but assume here diagonal psf. Could adjust this, but already very close to diagonal
kernel = ndimage.gaussian_filter(kernel, sigma=[np.sqrt(psf[2,2])/2.705078,np.sqrt(psf[0,0]),np.sqrt(psf[1,1])])

############

data = np.copy(dat) 
res = np.zeros(data.shape)
ndim = 3 #data.ndim 
algo = fd_restoration.RichardsonLucyDeconvolver(ndim, pad_mode='none', pad_min=[16,16,16]).initialize() 
tmp = algo.run(fd_data.Acquisition(data=data[0], kernel=kernel), niter=2)
for tt in range(SizeT):
    if tt%10==0:
        print(str(tt)+' of ' + str(SizeT) + ' complete')
    nonzero = data[tt][np.where(data[tt]>0)]
    bkgd_mode = stats.mode(nonzero)[0][0]
    bkgd_std = stats.tstd(nonzero)
    indz,indy,indx = np.where(data[tt]==0)
    data[tt,indz, indy, indx] = bkgd_mode + bkgd_std*np.random.randn(len(indz))
    # Note that deconvolution initialization is best kept separate from 
    # execution since the "initialize" operation corresponds to creating 
    # a TensorFlow graph, which is a relatively expensive operation and 
    # should not be repeated across iterations if deconvolving more than 
    # one image
    res[tt] = algo.run(fd_data.Acquisition(data=data[tt], kernel=kernel), niter=niter).data

####################
print('Done. Now writing to file\n')
# create numpy array with correct order
if is_single_time_pt:
    img5d = np.expand_dims(np.expand_dims(res, axis=1), axis=0) 
else:
    img5d = np.expand_dims(res, axis=1)
# write file and save OME-XML as description
tifffile.imwrite(output_file_str, img5d, metadata={'axes': dimorder}, description=xml)
print('All finished\n')